<a href="https://colab.research.google.com/github/nel417/Bert_Sentiment/blob/main/BERT_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision torchaudio

     |████████████████████████████████| 1.9MB 5.0MB/s 


In [4]:
!pip3 install transformers

     |████████████████████████████████| 2.3MB 4.9MB/s 
     |████████████████████████████████| 901kB 28.6MB/s 
     |████████████████████████████████| 3.3MB 40.0MB/s 


In [5]:
import torch
import requests
from bs4 import BeautifulSoup
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [6]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [24]:
#pt = pytorch
tokens = tokenizer.encode("I am happy, this is amazing. Wonderful Product. Absolutely incredible! I will be buying again. The best!", return_tensors='pt')
tokens[0]

tensor([  101,   151, 10345, 19308,   117, 10372, 10127, 39854,   119, 49108,
        20058,   119, 35925, 10563, 81981,   106,   151, 11229, 10346, 95909,
        12590,   119, 10103, 11146,   106,   102])

In [25]:
tokenizer.decode(tokens[0])

'[CLS] i am happy, this is amazing. wonderful product. absolutely incredible! i will be buying again. the best! [SEP]'

In [26]:
result = model(tokens)

In [30]:
# 0 - 4.
#0 is bad, 4 is incredible
result.logits

tensor([[-2.3112, -2.8776, -1.5875,  0.8234,  4.9150]],
       grad_fn=<AddmmBackward>)

In [29]:

torch.argmax(result.logits)

tensor(4)

In [31]:
r = requests.get('https://www.yelp.com/biz/the-coffee-ethic-springfield?osq=the+coffee+ethic')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class' :regex})
reviews = [result.text for result in results]

In [38]:
reviews

["The Coffee Ethic is one of the top choices in Springfield MO. Why did I say that? This is what happened. We saw tons of good reviews on Yelps so we decided to give it a try. They are under major construction, so we got a lavender latte to go. The latte is so delicious that we finished it in less than five minutes. So we go back immediately to grab another one, and order a bag of their Ethiopian beans so we can brew some at home as well. And they gave me a drip coffee to go for FREE because I bought a bag of bean. That's pretty cool!They have a few outdoor seatings, I guess if I had more time, I would slow down and chill for a bit, probably bring my dog with me as well.We have been doing pour over with their Ethiopian beans for a couple of days now, it's been great. The coffee is light roasted so you know it has to be in good quality.",
 "Definitely one of the best coffee shops in the area. \xa0We don't come by that often but we are always on the lookout for coffee shops wherever we g

In [40]:
import pandas as pd
import numpy as np

In [41]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [42]:
df

,review
0,The Coffee Ethic is one of the top choices in ...
1,Definitely one of the best coffee shops in the...
2,"It was a reasonably nice, aesthetically orient..."
3,My favorite coffee spot in Springfield! Locate...
4,"My cortado was well done, smooth. However, my ..."
5,The biggest thing I need from a coffee shop is...
6,Nice people. Good beverages. Prices moderately...
7,"Hipsters brew fantastic coffee, and I found th..."
8,When I'm in town for work I like to stop by he...
9,i am ENTHRALLED with this place the coffee ma...


In [44]:
def score(review):
  tokens = tokenizer.encode(review,return_tensors='pt')
  result = model(tokens)
  return torch.argmax(result.logits)

In [47]:
# 0 should be 4(great) 1 should be 4 (great), 2 should be 1 or 2(meh)
score((df['review'].iloc[0]))

tensor(4)

In [48]:
score((df['review'].iloc[1]))

tensor(4)

In [49]:
score((df['review'].iloc[2]))

tensor(2)

In [51]:
df['sentiment'] = df['review'].apply(lambda x: score(x[:512]))

In [53]:
df

,review,sentiment
0,The Coffee Ethic is one of the top choices in ...,tensor(4)
1,Definitely one of the best coffee shops in the...,tensor(4)
2,"It was a reasonably nice, aesthetically orient...",tensor(2)
3,My favorite coffee spot in Springfield! Locate...,tensor(4)
4,"My cortado was well done, smooth. However, my ...",tensor(2)
5,The biggest thing I need from a coffee shop is...,tensor(4)
6,Nice people. Good beverages. Prices moderately...,tensor(3)
7,"Hipsters brew fantastic coffee, and I found th...",tensor(3)
8,When I'm in town for work I like to stop by he...,tensor(3)
9,i am ENTHRALLED with this place the coffee ma...,tensor(4)


In [ ]:
# great great meh great meh great good good good great